In [2]:
import os
import tempfile
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
import math
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import StringLookup
from tensorflow import keras
from tensorflow.keras import regularizers, layers, models, regularizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (
    Dense, 
    Input, 
    concatenate,
    Flatten,    
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input

In [3]:
train_dir = 'data/train'
test_dir = 'data/test'
# 비디오 파일 목록과 태그를 포함하는 리스트를 만드는 함수
def create_data_list(data_dir):
    data_list = []
    # data_dir 안의 각 디렉토리에 대해 반복
    for item in os.listdir(data_dir):
        item_path = os.path.join(data_dir, item)  # 아이템의 전체 경로
        # 해당 경로가 디렉토리인지 확인
        if os.path.isdir(item_path):
            # 디렉토리 내의 모든 파일을 나열
            for file_name in os.listdir(item_path):
                # 파일이 .mp4 파일인지 확인
                if file_name.endswith('.jpg'):
                    # 리스트에 태그와 파일 경로를 추가
                    data_list.append((item, str(data_dir+'/'+item)+'/'+file_name))
    return data_list

# 함수를 사용해서 리스트를 생성
train_list = create_data_list(train_dir)
test_list = create_data_list(test_dir)
# 리스트에서 데이터프레임을 생성
train_df = pd.DataFrame(data=train_list, columns=['tag', 'image_name'])
test_df = pd.DataFrame(data=test_list, columns=['tag', 'image_name'])
# 필요한 경우 열 순서를 수정
train_df = train_df.loc[:, ['tag', 'image_name']]
test_df = test_df.loc[:, ['tag', 'image_name']]
# 데이터프레임을 CSV 파일로 저장
train_file_path = 'train.csv'
test_file_path = 'test.csv'
train_df.to_csv(train_file_path, encoding='utf-8-sig', index=False)
test_df.to_csv(test_file_path, encoding='utf-8-sig', index=False)
train_df = pd.read_csv("train.csv") 
test_df = pd.read_csv("test.csv")
print(f"Total video for training: {len(train_df)}")
print(f"Total video for testing: {len(test_df)}")

Total video for training: 160
Total video for testing: 40


In [4]:
# 손가락 각도 계산 함수
def calculate_angles(hand_landmarks, image_shape):
    joint = np.zeros((21, 3))
    for j, lm in enumerate(hand_landmarks.landmark):
        joint[j] = [lm.x * image_shape[1], lm.y * image_shape[0], lm.z]
    
    # 벡터 계산
    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]
    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]
    v = v2 - v1
    # 정규화
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

    # 각도 계산
    angle = np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
    angle = np.degrees(angle)
    
    return angle

In [4]:
# 이미지로부터 학습 데이터 생성 및 DataFrame에 추가하는 함수
def create_training_data_from_image(image_path, df):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
    
    # 이미지 파일 불러오기
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)

    # 양손의 랜드마크가 감지되었는지 확인하고 데이터 처리
    # 각 손에 대해 21개의 랜드마크에서 15개의 각도를 계산하므로 총 30개의 각도 데이터가 필요
    hand_data_row = [0] * 30  # 기본값으로 0으로 채워진 리스트를 생성
    if result.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(result.multi_hand_landmarks):
            # 각도 계산
            angles = calculate_angles(hand_landmarks, img.shape[:2])
            # 각 손에 대한 데이터를 적절한 위치에 넣기
            hand_data_row[i*15:(i+1)*15] = angles

    # 이미지 파일 경로에서 라벨(직전 폴더 이름) 추출
    label = os.path.basename(os.path.dirname(image_path))

    # DataFrame에 행 추가
    df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)

    return df

# 빈 DataFrame 초기화
df_hands = pd.DataFrame()

for path in train_df["image_name"].values.tolist():
    df_hands = create_training_data_from_image(path, df_hands)

# 컬럼 이름 설정 (30개의 각도 + 1개의 라벨)
angle_columns = [f'angle_{i}' for i in range(30)]
df_hands.columns = angle_columns + ['label']

# 모든 데이터가 추가된 DataFrame을 CSV 파일로 저장
df_hands.to_csv('training_data.csv', index=False)

C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(hand_data_row + [label]), ignore_index=True)
C:\Users\SMHRD\AppData\Local\Temp\ipykernel_42636\2512946801.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

In [5]:
df_hands = pd.read_csv("training_data.csv")

In [6]:
df_hands

,angle_0,angle_1,angle_2,angle_3,angle_4,angle_5,angle_6,angle_7,angle_8,angle_9,...,angle_21,angle_22,angle_23,angle_24,angle_25,angle_26,angle_27,angle_28,angle_29,label
0,13.538520,3.366123,13.208156,46.323339,54.103914,21.696195,0.517605,32.241149,4.799307,3.665180,...,5.683785,5.774647,0.485865,8.168108,3.185920,7.398080,22.146755,2.362355,2.071526,boa
1,19.752488,2.149095,19.116738,47.713572,40.770892,14.015137,4.404948,4.015291,0.474289,7.375577,...,2.301014,27.377630,1.384962,4.096881,16.829661,4.570777,23.572955,11.934741,7.543602,boa
2,20.139025,6.702180,11.439818,43.489300,56.094791,23.964255,5.278815,11.189080,4.976801,2.150516,...,6.269995,5.474323,1.441984,7.307729,2.543837,6.454899,19.912615,2.225977,3.313255,boa
3,24.793754,8.264628,14.733128,30.122039,41.840681,23.687930,1.447999,3.988998,3.034973,4.275817,...,5.071398,1.350209,5.416294,6.286543,5.848530,1.349553,20.548625,6.860131,5.561598,boa
4,25.328465,2.402904,6.511573,33.820285,50.757881,27.959092,2.024418,1.063441,7.176712,5.655424,...,3.900999,3.139319,3.626346,5.794378,0.390248,6.660261,16.331579,0.348413,5.149469,boa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2.100263,116.664988,8.846912,171.687640,42.314139,31.939439,172.177699,67.570072,68.871915,145.880948,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,you
156,15.931947,38.462765,38.514180,72.342186,7.718805,6.951463,121.761507,46.654680,136.087249,147.439473,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,you
157,25.203922,48.517827,31.780866,78.756453,13.305832,77.726892,95.931792,43.154833,131.388345,72.046227,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,you
158,22.304279,50.416759,35.745394,57.931212,21.919060,32.192960,95.489226,46.269301,118.357695,37.803634,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,you


In [9]:
y_train.value_counts()

boa       16
eat       16
go        16
jasae     16
me        16
meet      16
ore       16
pretty    16
sarang    16
you       16
Name: label, dtype: int64

In [7]:
X_train = df_hands.drop('label', axis =1)
y_train = df_hands['label']

In [10]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [13]:
# 필요한 Mediapipe 솔루션을 초기화합니다.
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        print("Ignoring empty camera frame.")
        continue

    # 이미지 처리
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks:
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 3))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z]

            # 벡터 계산
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]
            v = v2 - v1
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # 각도 계산
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:],
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
            angle = np.degrees(angle)

            # NaN 값이 있을 경우 0으로 대체
            angle = np.nan_to_num(angle)

            # 데이터 차원 맞추기
            data = np.concatenate((angle, [0]*(30-len(angle)))) if len(angle) < 30 else angle
            data = data.reshape(1, -1)  # 예측을 위해 데이터를 적절한 형태로 변환
            predicted_label = knn.predict(data)

            # 예측된 라벨을 화면에 표시
            org = (int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0]))
            cv2.putText(img, text=predicted_label[0], org=(org[0], org[1] + 20), 
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

            # 손 랜드마크 그리기
            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

    # 화면에 이미지 표시
    cv2.imshow('img', img)
    if cv2.waitKey(30) == 49:  # 1번 키를 누르면 종료
        break

# 사용 종료 후 자원 해제
cap.release()
cv2.destroyAllWindows()

C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWa

C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWa

C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWa

C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWa

C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWa

...

C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWa

C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWa

C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWa

C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWa

C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWa